In [1]:
#!pip install plotly
import pandas as pd
import numpy as np

In [6]:
data = pd.read_parquet('../data/light-curves/predict_minirocket_classifier.parquet')
data.head()

,oid,flare_start,LogReg_proba_class_0,LogReg_proba_class_1,RF_proba_class_0,RF_proba_class_1
index,,,,,,
0,801216200045207,58652.35533,0.998815,0.001185,0.974935,0.025065
1,801216200046771,58652.35759,0.955413,0.044587,0.778796,0.221204
2,801216200007518,58652.35532,0.980372,0.019628,0.975563,0.024437
3,801216200034901,58652.35531,0.955662,0.044338,0.867158,0.132842
4,801216200044860,58652.35531,0.998002,0.001998,0.953440,0.046560


In [10]:
import plotly.graph_objects as go

x = []
y_logreg = []
y_rf = []
y_union = []

for step in np.arange(0.0, 0.5,0.01):
    x.append(step)
    y_logreg.append(data[data.LogReg_proba_class_1 >= (1-step)].shape[0])
    y_rf.append(data[data.RF_proba_class_1 >= (1-step)].shape[0])
    y_union.append(data[((data.LogReg_proba_class_1 >= (1-step)) & (data.RF_proba_class_1 >= (1-step)))].shape[0])

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_logreg,
        mode='lines+markers',
        name='LogReg classifier'
    )
)
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_rf,
        mode='lines+markers',
        name='RF classifier'
    )
)
fig.add_trace(
    go.Scatter(
        x=x,
        y=y_union,
        mode='lines+markers',
        name='Union classifier'
    )
)

fig.update_layout(
    title='count of objects VS threshold',
    xaxis_title='threshold',
    yaxis_title='count of objects')
fig.show()

In [43]:
def samples_bin(df, name_model, threshold_start, threshold_end, n_bins, n_samples, random_state=42):
    result = pd.DataFrame(columns=['name_model', 'n_bin', 'borders', 'oid', 'flare_start', 'threshold'])
    col = f'{name_model}_proba_class_1'
    current_threshold = threshold_start
    step_threshold = (threshold_start - threshold_end)/n_bins
    n_bin = 1
    while current_threshold >= threshold_end:
        borders = (current_threshold, current_threshold - step_threshold)
        if name_model == 'Union':
            samples = df[
                (((df['LogReg_proba_class_1']<=borders[0])
                    & (df['LogReg_proba_class_1']>borders[1]))
                & ((df['RF_proba_class_1']<=borders[0])
                    & (df['RF_proba_class_1']>borders[1])))
            ]
        else:
            samples = df[
                ((df[col]<=borders[0])
                & (df[col]>borders[1]))
            ]

        if samples.shape[0] > 0:
            samples = samples.sample(n=n_samples, random_state=random_state)

            for _, row in samples.iterrows():
                if name_model == 'Union':
                    threshold = f"{row['LogReg_proba_class_1']}/{row['RF_proba_class_1']}"
                else:
                    threshold = str(row[col])

                result.loc[result.shape[0],:] = [
                    name_model, n_bin, str(borders),
                    int(row['oid']), row['flare_start'], threshold
                ]

        n_bin += 1
        current_threshold = current_threshold - step_threshold

    return result

In [45]:
models = ['LogReg', 'RF', 'Union']

for model in models:
    samples = samples_bin(data, model, threshold_start=1, threshold_end=0.9, n_bins=10, n_samples=10)
    samples.to_csv(f'../data/light-curves/{model}_10_samples_by_bin.csv')